In [ ]:
# Load packages
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import array_to_img
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from datetime import datetime

In [ ]:
# Mount (if using) google drive to the google colab jupyter notebook
drive.mount('/content/drive',force_remount=True)

In [ ]:
# Define path here
myPath = "drive/MyDrive/Dataset_group_50/"

# Define parameters here
IMAGE_SIZE = 256
BATCH_SIZE = 4
NUM_CLASSES = 9
TEST_SIZE = 0.2
RANDOM_SPLIT = 42
LEARNING_RATE = 0.0001
DATA_DIR = myPath

In [ ]:
# Test dataset is loaded correctly by displaying a image
img = cv2.imread(myPath + "data/Images/57_a.png")
cv2_imshow(img)
# or
# cv2.imshow(img)

In [ ]:
# Load training and valiation dataset
X = sorted(glob(os.path.join(DATA_DIR, "data/Images/*")))
y = sorted(glob(os.path.join(DATA_DIR, "data/Category_ids/*")))

# Load test dataset
test_images = sorted(glob(os.path.join(DATA_DIR, "data/Test_data/Images/*")))
test_masks = sorted(glob(os.path.join(DATA_DIR, "data/Test_data/Category_ids/*")))

# Add mirrored data
X = X + (sorted(glob(os.path.join(DATA_DIR, "mirrored/images/*"))))
y = y + (sorted(glob(os.path.join(DATA_DIR, "mirrored/labels/*"))))
# Add gaussianblur data
X = X + (sorted(glob(os.path.join(DATA_DIR, "gaussianblur/images/*"))))
y = y + (sorted(glob(os.path.join(DATA_DIR, "gaussianblur/labels/*"))))
# Add contrast-50 data
X = X + (sorted(glob(os.path.join(DATA_DIR, "contrast-50/images/*"))))
y = y + (sorted(glob(os.path.join(DATA_DIR, "contrast-50/labels/*"))))
# Add contrast+50 data
X = X + (sorted(glob(os.path.join(DATA_DIR, "contrast+50/images/*"))))
y = y + (sorted(glob(os.path.join(DATA_DIR, "contrast+50/labels/*"))))
# Add brightness-50 data
X = X + (sorted(glob(os.path.join(DATA_DIR, "brightness-50/images/*"))))
y = y + (sorted(glob(os.path.join(DATA_DIR, "brightness-50/labels/*"))))
# Add brightness+50 data
X = X + (sorted(glob(os.path.join(DATA_DIR, "brightness+50/images/*"))))
y = y + (sorted(glob(os.path.join(DATA_DIR, "brightness+50/labels/*"))))
# Add sharpen data
X = X + (sorted(glob(os.path.join(DATA_DIR, "sharpen/images/*"))))
y = y + (sorted(glob(os.path.join(DATA_DIR, "sharpen/labels/*"))))

train_images, val_images, train_masks, val_masks = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SPLIT)

print("Number of training images: " + str(len(train_images)))
print("Number of training masks: " + str(len(train_masks)))
print("Number of validation images: " + str(len(val_images)))
print("Number of validation masks: " + str(len(val_masks)))
print("Number of test images: " + str(len(test_images)))
print("Number of test masks: " + str(len(test_masks)))

def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    return image


def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask


def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset


train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)
test_dataset = data_generator(test_images, test_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)
print("Test Dataset:", test_dataset)

In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [ ]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)  
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
model.summary()

In [ ]:
# Define loss
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# Compile model with choosen optimizer 
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=loss,
    metrics=["accuracy"],
)

# Train model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=25)

# Create plots
plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()

In [ ]:
# Save model
path = "drive/MyDrive/carseg_data/" + datetime.now().strftime("%m-%d-%Y_%H-%M-%S") + "_model"
model.save(path)

In [ ]:
# To create segmentation with overlays download color palette
!gdown https://drive.google.com/uc?id=1B9A9UCJYMwTL4oBEo4RZfbMZMaZhKJaz
!unzip -q instance-level-human-parsing.zip

In [ ]:
# Loading the Colormap
# colormap = loadmat(
#     "./instance-level_human_parsing/instance-level_human_parsing/human_colormap.mat"
# )["colormap"]
# colormap = colormap * 100
# colormap = colormap.astype(np.uint8)

def infer(model, image_tensor):
    predictions = model.predict(np.expand_dims((image_tensor), axis=0))
    predictions = np.squeeze(predictions)
    predictions = np.argmax(predictions, axis=2)
    return predictions


def decode_segmentation_masks(mask, colormap, n_classes):
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    for l in range(0, n_classes):
        idx = mask == l
        r[idx] = colormap[l, 0]
        g[idx] = colormap[l, 1]
        b[idx] = colormap[l, 2]
    rgb = np.stack([r, g, b], axis=2)
    return rgb


def get_overlay(image, colored_mask):
    image = tf.keras.preprocessing.image.array_to_img(image)
    image = np.array(image).astype(np.uint8)
    overlay = cv2.addWeighted(image, 0.35, colored_mask, 0.65, 0)
    return overlay


def plot_samples_matplotlib(display_list, figsize=(5, 3)):
    _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=figsize)
    for i in range(len(display_list)):
        if display_list[i].shape[-1] == 3:
            axes[i].imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        else:
            axes[i].imshow(display_list[i])
    plt.show()


def plot_predictions(images_list, colormap, model):
    for image_file in images_list:
        image_tensor = read_image(image_file)
        prediction_mask = infer(image_tensor=image_tensor, model=model)
        prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 9)
        overlay = get_overlay(image_tensor, prediction_colormap)
        plot_samples_matplotlib(
            [image_tensor, overlay, prediction_colormap], figsize=(18, 14)
        )

In [ ]:
# Plot 20 predections using images from the test set
plot_predictions(test_images[:20], colormap, model=model)

In [ ]:
# Evaulate model using the inbuilt method.
model.evaluate(test_dataset)

In [ ]:
# Evaluate using DICE score
DICE_SCORES = []

for k in range(len(test_images)):
  image_tensor = read_image(test_images[k])
  label_tensor = cv2.imread(test_masks[k])
  label_tensor = label_tensor[:,:,0].flatten()
  prediction_mask = infer(image_tensor=image_tensor, model=model)
  prediction_mask = prediction_mask.flatten()

  TP = [0]*9
  NTP = [0]*9
  DICE = []
  for i in range(len(prediction_mask)):
      if prediction_mask[i] == label_tensor[i]:
          TP[prediction_mask[i]] = TP[prediction_mask[i]]+1

      else:
          NTP[prediction_mask[i]] = NTP[prediction_mask[i]]+1

  for j in range(len(TP)):
    smooth = 1000
    DICE.append((2*TP[j]+smooth)/(2*TP[j] + NTP[j]+smooth))
  DICE_SCORES.append(DICE)

DICE_MATRIX = np.array(DICE_SCORES)

print("Background: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[0]*100),"%")
print("Front Door: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[1]*100),"%")
print("Back Door: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[2]*100),"%")
print("Front Fender: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[3]*100),"%")
print("Body: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[4]*100),"%")
print("Front Bumper: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[5]*100),"%")
print("Hood: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[6]*100),"%")
print("Rear Bumper: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[7]*100),"%")
print("Trunk: ", "%.2f" % (DICE_MATRIX.mean(axis=0)[8]*100),"%")
print("Combined DICE score: ","%.2f" % (np.mean(DICE_MATRIX.mean(axis=0))*100),"%")



In [ ]:
# Save Dice score
np.save('drive/MyDrive/carseg_data/Gabriel_dataset_3_0_history_test.npy', history.history) 